
### Requisitos para o arquivo times.json

Objetivo: realizar processo de ETL e disponibilizar os dados em camada adequada.

1. Ler o arquivo json do Datalake [ok]
2. Renomear colunas para português usando snake case [ok]
3. Definir a tipagem de colunas correta (manualmente) [ok]
4. Coluna com data de cosumo do arquivo [ok]
5. Salvar dados em formato parquet na camada adequada [ok]

In [1]:
%run ../../Configuração-e-Utilitários/Variaveis.ipynb

In [2]:
%run ../../Configuração-e-Utilitários/Funções.ipynb

### Leitura do arquivo e definição de schema

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, BooleanType

# Criando o schema dos dados
schema = StructType(
    fields=[
        StructField("id", StringType(), False),
        StructField("name", StringType(), False),
        StructField("code", StringType(), False),
        StructField("founded", DoubleType(), False),
        StructField("national", BooleanType(), False),
        StructField("logo", StringType(), False),
    ]
)

# Lendo o arquivo passando o schema como parâmetro
times_df = spark.read.schema(schema).json(f"{caminho_bronze}/times.json")

times_df.show()

+---+-------------------+----+-------+--------+--------------------+
| id|               name|code|founded|national|                logo|
+---+-------------------+----+-------+--------+--------------------+
|  6|             Brazil| BRA| 1914.0|    true|https://media-1.a...|
|118|              Bahia| BAH| 1931.0|   false|https://media-1.a...|
|119|      Internacional| INT| 1909.0|   false|https://media-2.a...|
|120|           Botafogo| BOT| 1904.0|   false|https://media-2.a...|
|121|          Palmeiras| PAL| 1914.0|   false|https://media-2.a...|
|122|             Parana| PAR| 1989.0|   false|https://media-1.a...|
|123|       Sport Recife| SPO| 1905.0|   false|https://media-2.a...|
|124|         Fluminense| FLU| 1902.0|   false|https://media-1.a...|
|125|    America Mineiro| AME| 1912.0|   false|https://media-3.a...|
|126|          Sao Paulo| PAU| 1930.0|   false|https://media-2.a...|
|127|           Flamengo| FLA| 1895.0|   false|https://media-2.a...|
|128|             Santos| SAN| 191

In [6]:
# Definindo o tipo da coluna founded como integer. Se transformar direto no schema, os valores da coluna são perdidos.
times_df = times_df.withColumn("founded", times_df.founded.cast("integer"))

times_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- code: string (nullable = true)
 |-- founded: integer (nullable = true)
 |-- national: boolean (nullable = true)
 |-- logo: string (nullable = true)



### Alterando valores booleanos da coluna 'national' para 'Sim'/'Não' 

In [7]:
from pyspark.sql.functions import when

times_df = times_df.withColumn('national', times_df.national.cast('string'))
times_df = times_df.withColumn('national', when(times_df.national == 'true', 'Sim').otherwise('Não'))

times_df.show()

+---+-------------------+----+-------+--------+--------------------+
| id|               name|code|founded|national|                logo|
+---+-------------------+----+-------+--------+--------------------+
|  6|             Brazil| BRA|   1914|     Sim|https://media-1.a...|
|118|              Bahia| BAH|   1931|     Não|https://media-1.a...|
|119|      Internacional| INT|   1909|     Não|https://media-2.a...|
|120|           Botafogo| BOT|   1904|     Não|https://media-2.a...|
|121|          Palmeiras| PAL|   1914|     Não|https://media-2.a...|
|122|             Parana| PAR|   1989|     Não|https://media-1.a...|
|123|       Sport Recife| SPO|   1905|     Não|https://media-2.a...|
|124|         Fluminense| FLU|   1902|     Não|https://media-1.a...|
|125|    America Mineiro| AME|   1912|     Não|https://media-3.a...|
|126|          Sao Paulo| PAU|   1930|     Não|https://media-2.a...|
|127|           Flamengo| FLA|   1895|     Não|https://media-2.a...|
|128|             Santos| SAN|   1

### Renomeando os atributos para português com uso do snake_case

In [10]:
times_df = times_df.withColumnRenamed('code', 'sigla') \
                           .withColumnRenamed('country', 'pais') \
                           .withColumnRenamed('founded', 'ano_de_fundacao') \
                           .withColumnRenamed('id', 'id_time') \
                           .withColumnRenamed('logo', 'link_imagem_logo') \
                           .withColumnRenamed('name', 'nome_do_time') \
                           .withColumnRenamed('national', 'nacional?')

times_df.show()

+-------+-------------------+-----+---------------+---------+--------------------+
|id_time|       nome_do_time|sigla|ano_de_fundacao|nacional?|    link_imagem_logo|
+-------+-------------------+-----+---------------+---------+--------------------+
|      6|             Brazil|  BRA|           1914|      Sim|https://media-1.a...|
|    118|              Bahia|  BAH|           1931|      Não|https://media-1.a...|
|    119|      Internacional|  INT|           1909|      Não|https://media-2.a...|
|    120|           Botafogo|  BOT|           1904|      Não|https://media-2.a...|
|    121|          Palmeiras|  PAL|           1914|      Não|https://media-2.a...|
|    122|             Parana|  PAR|           1989|      Não|https://media-1.a...|
|    123|       Sport Recife|  SPO|           1905|      Não|https://media-2.a...|
|    124|         Fluminense|  FLU|           1902|      Não|https://media-1.a...|
|    125|    America Mineiro|  AME|           1912|      Não|https://media-3.a...|
|   

In [11]:
times_df = adicionar_data_de_ingestao(
    df = times_df,
    nome_coluna = 'data_de_ingestao',
    add_tempo = -3,
    unidade_de_medida = 'horas'
)

times_df.show()

+-------+-------------------+-----+---------------+---------+--------------------+----------------+
|id_time|       nome_do_time|sigla|ano_de_fundacao|nacional?|    link_imagem_logo|data_de_ingestao|
+-------+-------------------+-----+---------------+---------+--------------------+----------------+
|      6|             Brazil|  BRA|           1914|      Sim|https://media-1.a...|      2024-05-01|
|    118|              Bahia|  BAH|           1931|      Não|https://media-1.a...|      2024-05-01|
|    119|      Internacional|  INT|           1909|      Não|https://media-2.a...|      2024-05-01|
|    120|           Botafogo|  BOT|           1904|      Não|https://media-2.a...|      2024-05-01|
|    121|          Palmeiras|  PAL|           1914|      Não|https://media-2.a...|      2024-05-01|
|    122|             Parana|  PAR|           1989|      Não|https://media-1.a...|      2024-05-01|
|    123|       Sport Recife|  SPO|           1905|      Não|https://media-2.a...|      2024-05-01|


### Salvando arquivo

In [13]:
times_df.write.mode('overwrite').parquet(f"{caminho_silver}/times")

In [14]:
dbutils.notebook.exit('O arquivo "times.json" foi processado e carregado na camada silver.')